# <mark> Root Unique_ID - Onde Tudo Começa </mark>

 Atual notebook com as funçoes para processamento de PDF Pesquisavel e Raster e a criaçao dos Dataframes de forma dependente e unica

### Modules & config

In [1]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

# Modulos da solucao
import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron

In [2]:
#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


####Config Processamento Pipeline

# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 7. Path para DFs e CSVs exportados
export_path = "pipeline_extracao_documentos/6_geral_administacao/exports"

# 8. Path para lixeira
root_garbage_path = "pipeline_extracao_documentos/0_lixeira"



# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

#Modelo atual
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

# definindo localizadcao para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")

'pt_BR.utf8'

## 1. POC Unique_ID

### P1 - Funcoes para e-mail e extracao

In [3]:
# 1. Geraçao do hash do arquivo
def generate_file_hash(file_path):
    # Abre o arquivo em modo de leitura de bytes
    with open(file_path, "rb") as f:
        # Lê o conteúdo do arquivo
        file_data = f.read()
        # Utiliza o algoritmo SHA-256 para gerar o hash
        file_hash = hashlib.sha256(file_data).hexdigest()
    return file_hash

# 2. Geraçao do Unique_id do arquivo
def generate_unique_id():
    return str(uuid.uuid4())

# 3. XXX Busca proximo Batch
def busca_proximo_batch():
    # Abre o arquivo Excel e lê a coluna 'batch'
    df = pd.read_excel("pipeline_extracao_documentos/6_geral_administacao/exports/df_documento_recebido.xlsx", usecols=["batch"])
    # Pega o último valor da coluna 'batch'
    last_value = df.iloc[-1, 0]
    
    # Extraí o número do último batch e adiciona 1 para o próximo
    last_number = int(last_value.split("_")[1])
    next_number = last_number + 1
    
    # Forma o nome do próximo batch
    next_batch = f"Batch_{next_number}"
    
    return next_batch    

# 4. Função para verificar e criar a pasta se não existir
def check_and_create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
# 5. converte nome do arquivo
def conv_filename(title):
    
    # Divide o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remove acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substiti espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remove quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converte para minúsculas
    filename = filename.lower()

    # Adiciona a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 6. converte nome do arquivo retirando extensao
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename         

# 7. funçao que MOVE documentos e gera add_log_transaction_entry para df_log_transctions
def move_doc_processed_file(batch_name, src_path, tgt_path):
    
    function = "move_doc_processed_file"
    source_path = src_path
    file = os.path.basename(source_path)
    sub_dir = os.path.join(tgt_path, batch_name)
    destination_path = os.path.join(sub_dir, file)
    document_action = "move_processed_file"
    transaction_detail = (f'document {file} moved by: {function}')
    df_move = pd.DataFrame()
    try:
        document_unique_id = get_document_id_by_file(batch_name, file)
        check_and_create_folder(destination_path)
        shutil.move(source_path, destination_path)
        sucess = True
        move_log = add_log_transaction_entry(document_unique_id, batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess)
    except Exception as e:
        print(f"Erro ao mover documento: {e}")
        sucess = False
    
    return move_log    

# 8. Função para adicionar um novo registro em df_source
def add_source_entry(batch_name, file_path, file, type, level, parent_unique_id):
    #unique_id = generate_unique_id(type)
    unique_id = generate_unique_id()
    time_now = cron.timenow_pt_BR()   
    file_hash = generate_file_hash(file_path) 
    if level == 1:
        parent_unique_id = unique_id
    data = {
        'Batch': batch_name,
        'Data': time_now,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_ID': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }
    return data

# 9. Add nova linha para atualizar df_log_transctions
def add_log_transaction_entry(document_unique_id,batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess=True):

    data_log = {
        'Dt_Time': cron.timenow_pt_BR(),
        'Batch': batch_name,
        'File' : file,
        'Unique_ID': document_unique_id,
        'Action': document_action,
        'Sorce': src_path,
        'Target': tgt_path,
        'Transction_Detail': transaction_detail,
        'Sucess': sucess,    
    }
    
        
    return data_log


# 10. Consulta multiparametros
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id


# 12. Busca filhos - simples
def get_children(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Parent_Unique_ID=document_unique_id)


# 13. Busca pai -simples
def get_father(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    parent_unique_id = result['Parent_Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Unique_ID=parent_unique_id)


# 14. Pesquiso pai pelo Unique_ID e trago um dict
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }


# 15. Pesquiso pai pelo Unique_ID (document_parent_unique_id) e cria DICT
def get_father_by_unique_id(batch, document_parent_unique_id):
    
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=document_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }
    
# 16. Pesquiso pai pelo file do filho e cria DICT
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }        
        

# 17. Busca o 'Unique_ID' para definir o Parent_Unique_ID sem considerar 'Level'
def get_parent_unique_id(df_id_relations, batch_name, file, type):
    try:
        parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type)]['Unique_ID'].values[0]
    except IndexError:
        parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return parent_unique_id


# 18. funcao para trazer somente o 'Unique_ID'
def get_document_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Unique_ID'].values[0]
    except IndexError:
        document_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_unique_id


# 19. funcao para trazer somente o 'Parent_Unique_ID'
def get_document_parent_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Parent_Unique_ID'].values[0]
    except IndexError:
        document_parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_parent_unique_id


# 20. funçao para trazer toda a row de df_id_relations para o documento
def get_document_id_relations(df_id_relations, batch_name, file, type, level):
    try:
        document_id_relations = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)].values[0]
    except IndexError:
        document_id_relations = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_id_relations




# EXEMPLOS de Pesquisa DFss
    # get_document_unique_id(df_id_relations, batch_name, file, type, level)

    # # Busca somente o 'Parent_Unique_ID'
    # get_document_parent_unique_id(df_id_relations, batch_name, file, type, level)


    # #Busca todos os dados da row do documento encontrado
    # document_id_relations = get_document_id_relations(df_id_relations, batch_name, file, type, level)

    # document_batch = document_id_relations[0]
    # document_date = document_id_relations[1]
    # document_name = document_id_relations[2]
    # document_type = document_id_relations[3]
    # document_level = document_id_relations[4]
    # document_unique_id = document_id_relations[5]
    # document_parent_unique_id = document_id_relations[6]
    # document_hash = document_id_relations[7]
    # document_path = document_id_relations[8]

    # # Insercao de um registro pela func add_source_entry
    # file_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/SPA 15082023.rar"

    # file = os.path.basename(file_path)

    # type = "compressed_file_attachment"

    # level = 1

    # parent_unique_id = ''

    # # Adicionando um novo registro (substitua 'batch_name' e 'email' conforme necessário)
    # new_entry = add_source_entry(batch_name, file_path, file, type, level, parent_unique_id)

    # df_id_relations = df_id_relations.append(new_entry, ignore_index=True)

    # df_id_relations


# Busca proximo Batch caso nao esteja rodando email
batch_name = busca_proximo_batch()

# 1. Criaçao do DataFrame para armazenar as relações de Unique_ID e Parent_Unique_ID
df_id_relations = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'Parent_Unique_ID', 'Hash', 'File_Path'])

# 2. Criaçao do DataFrame para df_start_pipe:
#df_start_pipe = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'dt_hora', 'de', 'assunto', 'email', 'Hash'])

df_id_relations

,Batch,Data,File,Type,Level,Unique_ID,Parent_Unique_ID,Hash,File_Path


## 1. Implementando o Root Unique_ID

### <mark> <b>1.1</b> Funcoes importantes - algumas das quais estao sempre por ai  </mark>

In [7]:
# 2. XXX Analisa nro de paginas
def analisa_nro_pages(file_path):
    
    pdf_document = fitz.open(file_path)
    pages = pdf_document.pages() # generator object

    page_nro = []
    for page in pages:
        page_nro.append(page)
        
    nro_paginas = len(page_nro)    
    if nro_paginas > 1:
        doc_1_page = False
        return doc_1_page, nro_paginas    
    else:
        doc_1_page = True
        return doc_1_page, nro_paginas  
    pdf_document.close()
    
 

# XXX FUNCAO DE SPLIT
def split_documentos(qualquer_df, fase, atividade, status):
    
    documentos_splitados = []
    doc_info = {}
    rows_list = []
    documentos = []
    #output_dir = os.path.join(documentos_scan_path, batch_name)
    num_linhas_df = qualquer_df.shape[0]

    i = num_linhas_df + 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        nun_pages = row['pages']
        batch_name = row['batch']
        original_file_name = row['original_file_name']
        folder_name = row['directory']
        file_path = row['file_path']
        level = row['level']
        document_type = row['document_type']
        doc_action = row['doc_action']
        document_unique_id = idx
        new_level = level + 1
        
        if (doc_action == 'splitar') and (status == 'root_analise'):
            if nun_pages > 1:
                try:
                    pdf = fitz.open(file_path)
                    # Número total de páginas no PDF
                    total_pages = len(pdf)
                except Exception as e:
                    print(f"Nao congui abrir o PDF: {e}")    

                # Nome base para os arquivos de saída
                base_name = file_path.split('.')[0]  # Remove a extensão do arquivo
                file_to_delete = file_path
                # Loop para criar um novo PDF para cada página
                for page_num in range(total_pages):
                    # Cria um novo objeto PDF
                    new_pdf = fitz.open()
                    # Adiciona a página atual ao novo PDF
                    new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
                    # Nome do novo arquivo PDF
                    new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
                    # Salva o novo PDF
                    new_pdf.save(new_pdf_name)
                    # Fecha o novo PDF
                    new_pdf.close()
                    rotulo = "prov_nota_fiscal"
                    acao_sugerida = sugestoes_acao.get(rotulo, "no_defined_action")
                    acao_executada = "novo_doc_criado"
                    informations = (f'documento criado a partir do split do documento: {original_file_name}')  
                    name_pdf_splited = os.path.basename(new_pdf_name)

                    new_row = {
                        "seq": i,
                        "date_time": cron.timenow_pt_BR(),
                        "batch": batch_name,
                        "fase_processo": fase,
                        "nome_atividade": atividade,
                        "status_documento": status,
                        "acao_executada": acao_executada,
                        "original_file_name": new_pdf_name,
                        "directory": folder_name,
                        "one_page": True,
                        "pages": 1,
                        "document_type": rotulo,
                        "doc_action": acao_sugerida,
                        "level": level,
                        "document_unique_id": generate_unique_id(),
                        "parent_document_unique_id": document_unique_id,
                        "file_hash": generate_file_hash(file_path),
                        "file_path": file_path,
                        "informations": informations,
                    }
                    rows_list.append(new_row)
                    i += 1
                qualquer_df.loc[idx, 'status_documento'] = "NAO_PROCESSAR" 
                qualquer_df.loc[idx, 'informations'] = "Paginas splitada em multiplos documentos" 
                qualquer_df.loc[idx, 'date_time'] = cron.timenow_pt_BR() 
    
    total_split = i - 1        
    df_split = pd.DataFrame(rows_list)
    
    
    return df_split, rows_list


# XXX Usando na criacao da imagem 
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 


# XXX Funcao ajustada para convertere e resize
def convertResize(doc2convert, document_path, image_resized_path):
    
    
    name_image = conv_filename_no_ext(doc2convert)
    
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    #print(f'image_resized_name: {image_resized_name}\n')
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
        image_2work = resized_pages[0]
        
    return image_2work, image_resized_name



def apagar_zone(documentos_extracao_path):
    # Para apagar arquivos PDF:Zone
    for root, dirs, files in os.walk(documentos_extracao_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            #print(file)
            if ":Zone" in file:
                file_to_delete = file_path
                os.remove(file_to_delete)
                #print(file, "termina, pode eliminar")
                
                
def confirma_pdf_pequisavel(file_path):
    
    pdf_document = fitz.open(file_path)
    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    # Definir retângulo de interesse
    try:
        x0 = 0
        y0 = 4
        x1 = 600
        y1 = 200  # Ajuste este valor para delimitar a região vertical
        # Extrair texto dentro do retângulo
        text = page.get_text("text", clip=(x0, y0, x1, y1))
        if text:
            page_number = 0
            pdf_pequisavel = True
        #print(page_number)
        else:
            page_number = 1
            pdf_pequisavel = False
        #print(page_number)
    except Exception as e:
        msg_error = (f"Erro ao abrir pagina do PDF: {e}")
        pdf_pequisavel = False
        pdf_document.close()   
         
        return pdf_pequisavel
                   

#generated_parent_document_unique_id = generate_unique_id()  

# Processo de deleçao e atualizacao de documentos
#e_deleta_peloamor(df_docs_splitados)

#me_atualiza_logo_vai_2(novo_df)

# apagar_zone(documentos_extracao_path)

In [ ]:
fase = 'processamento' # 'pre-processamento'
atividade = 'pre_process'
status = 'root_analise'

acao_sugerida = "splitar"


df_root_pipe = split_documentos(df_root_pipe, fase, atividade, status)

In [ ]:
qualquer_df = pd.concat([qualquer_df, df_split], ignore_index=False)

### Template e dicts

In [55]:
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v9.xlsx"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"


#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

def define_dados_iniciais(texto_tratado):
    
    dados_iniciais_nf = {}

    prefeitura_encontrada = None
    de_para_encontrado = None

    # 7. ZZZ Dicionário para mapear Prefeitura com sua sigla
    de_para_prefeitura = {
        "PREFEITURA DA CIDADE MAGE": "PM_MAGE",
        "PREFEITURA DA CIDADE DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE MAGE": "PM_MAGE",
        "PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "MUNICIPAL DE SAO PEDRO DA ALDEIA": "PM_SPA",
        "PREFEITURA MUNICIPAL DE MESQUITA": "PM_MESQUITA",
        "PREFEITURA MUNICIPAL DE DE MESQUITA": "PM_MESQUITA",
        # ... adicione 
    }

    templates = {
        ("PM_MAGE", "30.693.231/0001-99"): "MAGE_MAICON",
        ("PM_MAGE", "23.317.112/0001-76"): "MAGE_MFF",
        ("PM_MAGE", None): "MAGE",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "47.945.459/0001-21"): "SAO_PEDRO_GOAT",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "68.687.722/0001-08"): "SAO_PEDRO_GM",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "34.230.979/0038-06"): "SAO_PEDRO_SUPERMIX",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", None): "SAO_PEDRO",
        ("Pague agora com o seu Pix", None): "NAO_PROCESSAR",
        # ... adicione outras combinações aqui
    }

    cnpj_encontrado = None
    # Verifique cada linha do texto
    for linha in texto_tratado:
        for pref in de_para_prefeitura.keys():
            if pref in linha:
                #print(linha)
                prefeitura_encontrada = pref
                dados_iniciais_nf['prefeitura'] = prefeitura_encontrada
                #print(prefeitura_encontrada)
    # Saímos do loop, agora vamos verificar qual template usar
    if prefeitura_encontrada:
        de_para_pm = de_para_prefeitura.get(prefeitura_encontrada)
        dados_iniciais_nf['de_para_pm'] = de_para_pm
        #print(de_para_pm)
        if not de_para_pm:
            de_para_pm = de_para_prefeitura.get(prefeitura_encontrada, "NAO_PROCESSAR")
            dados_iniciais_nf['de_para_pm'] = de_para_pm
            #print(de_para_pm)
    else:
        de_para_pm = "NAO_PROCESSAR"
        #print(de_para_pm)
        
        
        # Verifique cada linha do texto
    for linha in texto_tratado:
        for de_para, cnpj in templates.keys():
            if cnpj and cnpj in linha:
                cnpj_encontrado = cnpj
                dados_iniciais_nf['cnpj_encontrado'] = cnpj_encontrado
                print(cnpj_encontrado)
                
    # Saímos do loop, agora vamos verificar qual template usar
    if de_para_pm:
        template_usar = templates.get((de_para_pm, cnpj_encontrado))
        # print(template_usar)
        dados_iniciais_nf['template_usar'] = template_usar
        if not template_usar:
            template_usar = templates.get((de_para_pm, None), "TEMPLATE_NAO_ENCONTRADO")
            dados_iniciais_nf['template_usar'] = template_usar 
    else:
        template_usar = "TEMPLATE_NAO_ENCONTRADO"
        dados_iniciais_nf['template_usar'] = template_usar
        
    
    #Confirmando se template existe em frames    
    try:        
        f_type = 'frame'
        #template_usar = 'SAO_PEDRO_SUPERMIX'
        result = filtrar_df(frames_nf_v4_df, type=f_type, de_para_pm=de_para_pm, model=template_usar)
        model = result['model'].values[0]
        if model:
            template_oficial = model
            if model == template_usar:
                dados_iniciais_nf['model'] = template_oficial
            else:    
                template_usar = "necessario cadastrar"
                dados_iniciais_nf['template_usar']
                
            dados_iniciais_nf['template_usar'] = template_usar
        else:
            template_usar = "necessario cadastrar"
            dados_iniciais_nf['template_usar'] = template_usar
            dados_iniciais_nf['model'] = template_usar
                
    except Exception as e:
       error_msg = (f"Erro busca cnae: {e}") 
               
        
    return dados_iniciais_nf  



# Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "NO_PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}


def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'

palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)

print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

nome_arquivo:                              batatinha_quando_nasce.pdf | palavra_chave:              default | rotulo: prov_nota_fiscal     | acao_sugerida: NO_PROCESS                    


### Funcoes de OCR

In [58]:
# XXX Funcao de extracao
def extract_text_PIL(image, coordinates):
    x0, y0, x1, y1 = coordinates
    image_croped = image.crop((x0, y0, x1, y1))
    texto_extraido = pytesseract.image_to_string(image_croped, lang='por')
    return texto_extraido 





def extract_text_from_coordinates(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    print(x0, y0, x1, y1)
    frame_image
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

def extract_text_from_coordinates_2(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    print(x0, y0, x1, y1)
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config_1).strip()
    return extracted_text


def extract_fields_box(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame_2(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number2(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores


# 3. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

 
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text 


def extract_fb_outras_inf(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == model)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        
        string_pesquisa = row_field['reference']
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        label = row_field['label']
        #print(f'extracted_text_box {extracted_text_box}, label {label}')
        text = extracted_text_box.replace('\n', '')
        if text.startswith(string_pesquisa):
            #print("aqui:", text)
            text = text[len(label):].strip()
            data_box_valores[label] = text
    
    return   data_box_valores  


### Funcoes REGEX e ORganizacao TEXTO

In [14]:
def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

# Funçao de formatacao de numeros
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)



#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return None
    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return None
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return None
        else:
            return None
        
#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    else:
        return None  


    
def pesquisa_keyword(string_pesquisa, text_splited, keyword_list):

    resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)

    if resultado_extraido_fuzz == None:
        resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
        if resultado_extraido_frame_up == None:
            resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
            resultado_extraido = resultado_extraido_frame_down
        else:
            resultado_extraido = resultado_extraido_frame_up
    else:
        resultado_extraido = resultado_extraido_fuzz           
    #print(resultado_extraido)
    return resultado_extraido



 





def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = None
            
    
    return nf_data_tomador_cnpj 



def extract_dados_comple_obs(modelo, frame_father, section):
    
    data_dados_complementares = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            #print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))
            
            if frame_father == "5_frame_dados_complementares":
                nf_data_dados_complementares = {}
                nf_data_dados_complementares['section'] = section
                
                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^DADOS COMPLEMENTARES', '', extracted_text_box, count=1)
                if text == '':
                    text = None
                    nf_data_dados_complementares['dados_complementares'] = text
                else:    
                    # Extrair texto dentro do retângulo
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                    
                return nf_data_dados_complementares                
                
            elif frame_father == "5_frame_observacao":
                nf_data_observacao = {}
                nf_data_observacao['section'] = section 
                                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^Observação:', '', extracted_text_box, count=1)

                # Remover quebras de linha
                text = text.replace('\n', ' ')

                # Extrair texto dentro do retângulo
                nf_data_observacao['observacao'] = text.strip()
                
                return nf_data_observacao 
            




def processa_cnae_outros(text):
    nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
    if nf_data_CNAE_match:
        try:
            # Remove a primeira ocorrência de "CNAE:"
            nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
            # Remover quebras de linha
            nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
            return nf_data_CNAE_str 
        except Exception as e:
            print(f"Erro busca cnae: {e}") 
        
    return None   





## <mark> Analise de Extracao de dados no Pipeline </mark>

In [15]:
batch_name = "Batch_20" #Excepcionalmente

In [16]:
fake_parent_document_unique_id = generate_unique_id()

### Importantando df_root_pipe

In [ ]:
df_root_pipe = pd.DataFrame()

In [17]:
# 0. XXX SE deseja importar o DF df_analise_pipe

df_root_pipe_path = "df_root_analise.xlsx"


#Le a planilha e cria df_documento_recebido
df_root_pipe = pd.read_excel(df_root_pipe_path)

# Ajusta o indice
df_root_pipe.set_index('document_unique_id', inplace=True)

In [ ]:
df_root_pipe

###  2.2 Funçao de Analise do Pipeline - gerar <mark> <b>df_root_pipe </b></mark>

In [19]:
# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos - ESSA E A UNICA FUNCAO QUE ITERA NO DIRETORIO
def scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status):
    
    doc_info = {}
    resumo = {}
    raw_document = []
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            nome_arquivo = file
            palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)
            acao_executada = "Analise"
            informations = ' '    
            file_name = file.lower()    
            file_path = os.path.join(root, file)
            new_path_name = os.path.join(output_dir, file)
            if file.lower().endswith('.pdf'):
                doc_one_page, nro_pgs = analisa_nro_pages(file_path)
            # if doc_one_page == False:
            #             rotulo = 'pdf_mul_paginas'
            level = 3
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
                
            #print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    
            
            new_row = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "batch": batch_name,
                "fase_processo": fase,
                "nome_atividade": atividade,
                "status_documento": status,
                "acao_executada": acao_executada,
                "original_file_name": file,
                "directory": folder_name,
                "one_page": doc_one_page,
                "pages": nro_pgs,
                "palavra_chave": palavra_chave,
                "document_tag": rotulo,
                "action_item": acao_sugerida,
                "level": level,
                "document_unique_id": generate_unique_id(),
                "parent_document_unique_id": fake_parent_document_unique_id,
                "file_hash": generate_file_hash(file_path),
                "file_path": file_path,
                "informations": informations,
            }
            raw_document.append(new_row)

            
            # print(f'seq: {i} | file: {file}'
            i += 1
    df_trans_pipe = pd.DataFrame(raw_document)
      
                
    return df_trans_pipe, raw_document

In [ ]:
df_root_pipe

In [ ]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'root_analise'

documentos = []

#df_trans_pipe = pd.DataFrame()

df_root_pipe, documentos = scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')

In [ ]:
# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_root_pipe[df_root_pipe['one_page'] == False]

In [ ]:
df_root_pipe.set_index('document_unique_id', inplace=True)

### Incorporando o processo de IteraÇao

In [ ]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status):
    
    linhas_analise = []
    bloco_1_list = []
    bloco_2_list = []
    bloco_3_list = []
    imagens_list = []  
    new_data = [] 
    
    pre_processo = ['nro_nota', 'competencia', 'dt_hr_emissao', 'codigo_verificacao', 'ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura', 'enviar_canceladas', 'enviar_listagens']
    
    time_now = cron.timenow_pt_BR()
    
    fase_processo_atual = fase
    atividade_processo_atual = atividade
    status_documento_atual = status
    
    i = 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        
        # 1. Mapeamento de informacoes do DF
        document_unique_id = idx
        seq_df = row['seq']
        batch_name = row['batch']
        fase_processo = row['fase_processo']
        nome_atividade = row['nome_atividade']
        status_documento = row['status_documento']
        original_file_name = row['original_file_name']
        file_directory = row['directory']
        level = row['level']
        d_type = row['level']
        document_type = row['document_type']
        pdf_pesquisavel = row['pdf_pesquisavel']
        one_page_doc = row['one_page']
        modelo = row['modelo']
        
        prefeitura = row['prefeitura']
        
        parent_document_unique_id = row['parent_document_unique_id']
        file_path = row['file_path']
        file_hash = row['file_hash']
        
        # 2. Busca modelo
        if (not status_documento == 'NAO_PROCESSAR') or (not document_type == 'outros') or (not one_page_doc == True):
            
            if atividade_processo_atual == 'extracao_prestador':
            #print(f' 1 - seq: {seq_df} | file: {original_file_name} |status_documento: {status_documento} pdf_pesquisavel: {pdf_pesquisavel}\n')
                if status_documento == status_documento_atual:
         
                    print(f'seq_df: {seq_df} status_documento: {status_documento} | modelo: {modelo:>20} | file: {original_file_name:>40} | prefeitura: {prefeitura:>55} | {pdf_pesquisavel}\n')
                    
                    result_list = []
                    
                    dfcnpj_prestador = {}
                    dfincricao_prestador = {}
                    dfdados_prestador = {}
                        
                    # doc2convert = original_file_name
                    section = "2. PRESTADOR DE SERVIÇO"
                    modelo = 'SAO_PEDRO_SUPERMIX'
                    f_tipo = 'frame'
                    
                    dfcnpj_prestador, dfincricao_prestador, dfdados_prestador, textoextraido  = processar_dados_dados_documentos(row, original_file_name, file_path, pdf_pesquisavel, section, modelo, f_tipo)
                    
                    print(f'\n {dfcnpj_prestador}\n{dfincricao_prestador}\n{dfdados_prestador}\n')
                    
               
      


                i += 1
              
    return textoextraido

In [ ]:
df_root_pipe.to_excel('df_root_analise.xlsx', index=True)

### Se Quiser evoluir no doc Analise

In [ ]:
# 0. XXX SE deseja importar o DF df_analise_pipe

df_analise_pipe_path = "Em_HOLD/df_mapeamento_e_analise2.xlsx"


#Le a planilha e cria df_documento_recebido
df_analise_pipe = pd.read_excel(df_analise_pipe_path)

# Ajusta o indice
df_analise_pipe.set_index('document_unique_id', inplace=True)

In [ ]:
df_analise_pipe

In [ ]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'mapear'

df_analise_pipe, documentos = scan_pipeline_documentos(documentos_scan_path, batch_name, fase, atividade, status)

<mark> <b>2.x</b> Processo de mapeamento e criacao do  df_analise_pipe </mark>

In [ ]:

# 2. XXX df_analise_pipe e o df oficial de analise do pipeline
df_analise_pipe

# 3. Numero de linhas do DF
num_linhas_df = df_analise_pipe.shape[0]

num_linhas_df # variavel para o numero de linhas do DF


# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_analise_pipe[df_analise_pipe['one_page'] == False]

num_docs_2_split = df_pages_2_split.shape[0]


print(f'nro documentos em df_analise_pipe: {num_linhas_df} | nro documentos + 1 pagina: {df_pages_2_split.shape[0]}')
    
# 5. XXX Definimos o Index dos DFs
df_analise_pipe.set_index('document_unique_id', inplace=True)

df_pages_2_split.set_index('document_unique_id', inplace=True)



# 6. XXX Executo a criacao dos documentos split 
fase = 'pre-processamento'
atividade = 'scan_analise'
status = 'processar'

# df_docs_splitados recebera o DF com os documentos splitados
df_docs_splitados = split_documentos(df_pages_2_split, df_analise_pipe, num_linhas_df, fase, atividade, status)



# 7. XXX Retiro o indice do DF - Resetando o índice e mantendo o índice original como uma nova coluna
df_analise_pipe.reset_index(inplace=True)

# 8. XXX Concatenando os DataFrames
df_analise_pipe = pd.concat([df_analise_pipe, df_docs_splitados], ignore_index=True)

# 9. XXX Volto novamente o indice do DF
df_analise_pipe.set_index('document_unique_id', inplace=True)

# 10. XXX Salvo em Excel (pode ser feito durante fases)
df_analise_pipe.to_excel("df_mapeamento_e_analise.xlsx", index=True)

In [ ]:
# 2B. XXX Efetuo a analise do pipeline de documentos e inicio da extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'Reavaliar_PDF_Pesquisavel' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'mapear'

imagens_list = analise_extracao_pipeline(subset_df_analise_pipe, df_analise_pipe, fase, atividade, status)


if imagens_list:
    remove_images(imagens_list)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

### Ajustando dados do DF e efetuando queries

In [ ]:
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: row['coluna1'] * 2 if row['coluna2'] > 0 else row['coluna1'], axis=1)

In [ ]:
# Atualizei o DF baseado em condiçao de outra coluna
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: False if row['status_documento'] == "Template_encontrado" else row['pdf_pesquisavel'], axis=1)


# Podemos chegar de forma mais rapida
df_analise_pipe.loc[df_analise_pipe['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False


In [ ]:
subset_df_analise_pipe.quert('nro_nota')

In [ ]:
# Efetuando pesquisa no DF
df_analise_pipe.query('pdf_pesquisavel == False')

In [ ]:
df_analise_pipe.query('nro_nota == 13')

In [ ]:
# XXX Para Analise de PDF Pesquisavel, uei uma copia do df_analise_pipe
subset_df_analise_pipe = df_analise_pipe

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq < 21 & seq > 10')

In [ ]:
subset_df_analise_pipe

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=13, column='model_frame', value=None)

<mark> <b>1.</b> Efetuando subset do df para pesquisa </mark>

In [ ]:
# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df_analise_pipe = df_analise_pipe.iloc[:10]

subset_df_analise_pipe

In [ ]:
# 1. Usando loc para Filtrar Baseado em Condições
subset_df = df_scan_pipe.loc[df_scan_pipe['directory'] == 'camaleao']

subset_df

In [ ]:
assert file_hash == file_hash2, "Os arquivos são diferentes"

In [ ]:
# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df = df_scan_pipe.iloc[:10]

subset_df

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df = df_scan_pipe.query('directory == "root_dir" & seq < 10 & seq > 7')

subset_df

In [ ]:
# 4. Filtrando Linhas Baseadas em Valores em uma Lista

valores = [11, 16, 30, 41]
subset_df = df_scan_pipe[df_scan_pipe['seq'].isin(valores)]

subset_df

In [ ]:
registro_específico = df_scan_pipe.loc['e1f4b1af-30f3-45d2-85a7-1bb895bd5325']

registro_específico

## 2. Efetuando Extracao

In [20]:
# Funcoes de extracao
def processar_dados_iniciais(row, original_file_name, file_path):
    
    # lista_texto_extraido = []
    nf_dados_doc = {}
    pdf_pesquisavel = None
    extracted_txt = pesquisa_prefeitura_pdf_pesquisavel(file_path)
    
    if extracted_txt:
        pdf_pesquisavel = True
    else:
        pdf_pesquisavel = False 
       
        x0 = 110
        y0 = 0
        x1= 1929
        y1 = 786
        
        image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
        extracted_txt = extract_text_PIL(image_2work, (x0, y0, x1, y1))
    
    nf_dados_doc['file_name'] = original_file_name    
    nf_dados_doc['pdf_pesquisavel'] = pdf_pesquisavel 
    value = {}   
    texto_tratado = texto_extraido(extracted_txt)
    value = define_dados_iniciais(texto_tratado)
    if value:
        nf_dados_doc.update(value)
   


    return nf_dados_doc

def pesquisa_prefeitura_pdf_pesquisavel(file_path):    
    
    # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Definir retângulo de interesse
    x0 = 0
    y0 = 4
    x1 = 600
    y1 = 200  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    #print(text)
    
    if text:
       page_number = 0
       #print(page_number)
    else:
       page_number = 1
       #print(page_number)
    
    pdf_document.close()
   
    return text

In [51]:
# XXX Funcoes de Regex - cabecalho - documento ped pesquisavel
nf_data_servico = {}
nf_data_erros = {}
nf_lista_erros = []

# 0. Pesquisa PDF
def is_pdf_searchable(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        pdf_document.close()
        return is_searchable
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

# 1. CABECALHO
def extract_fields_cabecalho(text):
    nf_data_cabecalho = {}
    # Extrair Nome da Prefeitura
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', text)
    if nome_prefeitura_match:
        nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        nf_data_cabecalho['nome_prefeitura'] = nome_prefeitura
    else:
        nf_data_cabecalho['nome_prefeitura'] = None    

    # Extrair Tipo de NF
    tipo_nf_match = re.search(r'NOTA FISCAL (.+)', text)
    if tipo_nf_match:
        tipo_nf = "NOTA FISCAL " + tipo_nf_match.group(1)
        nf_data_cabecalho['tipo_nota_fiscal'] = tipo_nf
    else:
        nf_data_cabecalho['tipo_nota_fiscal'] = None    
    
    # Extrair Número da Nota
    numero_nota_match = re.search(r'Número da Nota:\s+(\d+)', text)
    if numero_nota_match:
        nr_nro_nf = numero_nota_match.group(1)
        nf_data_cabecalho['numero_nota_fiscal'] = numero_nota_match.group(1)
    else:
        nf_data_cabecalho['numero_nota_fiscal'] = None
        nf_data_erros['cabecalho'] = "numero_nota"
        nf_lista_erros.append(nf_data_erros)        

    # Extrair Competência
    competencia_match = re.search(r'Competência:\s+(.+)', text)
    if competencia_match:
        nf_data_cabecalho['competencia'] = competencia_match.group(1)
    else:
        nf_data_cabecalho['competencia'] = None
        nf_data_erros['cabecalho'] = "competencia"
        nf_lista_erros.append(nf_data_erros)
            

    # Extrair Data e Hora de Emissão
    data_emissao_match = re.search(r'Data e Hora da Emissão:\s+(.+)', text)
    if data_emissao_match:
        nf_data_cabecalho['dt_hr_emissao'] = data_emissao_match.group(1)
    else:
        nf_data_cabecalho['dt_hr_emissao'] = None    
        
    # Extrair Data e Hora de Emissão
    codigo_verificacao_match = re.search(r'Código Verificação:\s+(.+)', text)
    if codigo_verificacao_match:
        nf_data_cabecalho['codigo_verificacao'] = codigo_verificacao_match.group(1)
    else:
        nf_data_cabecalho['codigo_verificacao'] = None
        nf_data_erros['cabecalho'] = "codigo_verificacao"
        nf_lista_erros.append(nf_data_erros)        

    return nf_data_cabecalho


def processar_cabecalho_pdf_pesquisavel(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    
    section = "1 - CABECALHO"
    frame_type = 'frame'
    f_frame_label = "1_frame_dados_nf"
    data_box_valores = {'secao': section}
    nf_data_cabecalho = {}
    message_erro = []
    nf_data_cabecalho['original_file_name'] = original_file_name
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['type'] == frame_type) & (frames_nf_v4_df['label'] == f_frame_label)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        frame_id = row_frame['id']
        label = row_frame['label']
        x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
        #print(f'doc {"pdf_pesquisavel" if pdf_pesquisavel else "raster_pdf"}: {label:>30} | id: {frame_id:>3} | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6} ')
        
        pdf_document = fitz.open(file_path)
        # Página do PDF
        page_number = 0  # Defina o número da página que deseja analisar
        page = pdf_document[page_number]
        #x0, y0, x1, y1 = (0, 0, 600, 110)
        text = page.get_text("text", clip=(x0, y0, x1, y1))
        nf_data_cabecalho = extract_fields_cabecalho(text)
        # nf_data_cabecalho['original_file_name'] = original_file_name
        # nf_data_cabecalho['pdf_pesquisavel'] = pdf_pesquisavel_map
        
        pdf_document.close()
        
    return nf_data_cabecalho 



def processar_cabecalho_raster_pdf(row, pdf_pesquisavel_map, model_map, original_file_name, file_path):
    section = "1 - CABECALHO"
    pdf_pesquisavel = False
    father = "1_frame_prefeitura_nf"
    data_extrated_prefeitura = {}
    tipo_2 = "sframe_field"
    
    data_extrated_prefeitura['pdf_pesquisavel'] = pdf_pesquisavel_map
    data_extrated_prefeitura['original_file_name'] = original_file_name
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['section_json'] == section)]
    filtered_sframe_field_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_map) & (frames_nf_v4_df['father'] == father) & (frames_nf_v4_df['type'] == tipo_2)]
    image_2work, image_resized_name = convertResize(original_file_name, file_path, image_resized_path)
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        label_value = row_frame['label']
        reference_value = row_frame['reference']
        f_type = row_frame['type']
        frame_id = row_frame['id']
        if row_frame['label']  == "1_frame_prefeitura_nf":
            x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel_map else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            #print(texto_extraido)
            label_value = row_frame['label']
            #print("label_value", label_value)
            values = texto_extraido.split('\n')
            for index_sframe, row_sframe in filtered_sframe_field_nf_v4_df.iterrows():
                label_value = row_sframe['label']
                #print("label_value", label_value)
                if label_value == "nome_prefeitura":
                    reference_value = row_sframe['reference']
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result)
                elif label_value == "secretaria":
                    reference_value = row_sframe['reference']
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result) 
                elif label_value == "tipo_nota_fiscal":
                    reference_value = row_sframe['reference']  
                    for value in values:
                        result = process_line(value, reference_value, label_value)
                        if result:
                            data_extrated_prefeitura.update(result)
        
        if row_frame['label']  == "1_frame_dados_nf":
            x0, y0, x1, y1 = (row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']) if pdf_pesquisavel else (row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'])
            texto_extraido = extract_text_PIL(image_2work, (x0, y0, x1, y1))
            #print(texto_extraido)
            text_splited = texto_extraido_cabecalho(texto_extraido)
            keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

            string_pesquisa = "Número da Nota:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
            data_extrated_prefeitura['numero_nota_fiscal'] = texto

            string_pesquisa = "Competência:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_extrated_prefeitura['competencia'] = texto

            string_pesquisa = "ata e Hora da Emissão:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_extrated_prefeitura['dt_hr_emissao'] = texto

            string_pesquisa = "Código Verificação:"
            texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
            data_extrated_prefeitura['codigo_verificacao'] = texto
            #print(f'{frame_id:>5}  {f_type:>20} | {label_value:>30} |  ref:{reference_value:>30}  | x0: {x0:>6} y0: {y0:>6} x1: {x1:>6} y1: {y1:>6}')

    return data_extrated_prefeitura


In [59]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status):
    
    doc_info = {}
    resumo = {}
    row_teste_info = []
    
    time_now = cron.timenow_pt_BR()
    
    func_fase = fase
    func_atividade = atividade
    func_status = status
    lista_dicts = []
   
    i = 1
    for idx, row in qualquer_df.iterrows():
        dados_iniciais = {}
        # dados_root_pipe[idx] = row.to_dict()
        row_info = row.to_dict()
        message_erro = []
        # 1. Mapeamento de informacoes do DF
        map_document_unique_id = idx
        #dados_root_pipe['document_unique_id'] = map_document_unique_id
        
        map_seq = row['seq']
        #dados_root_pipe['seq'] = map_seq
        map_batch_name = row['batch']
        #dados_root_pipe['batch'] = map_batch_name
        map_fase_processo = row['fase_processo']
        #dados_root_pipe['fase_processo'] = map_fase_processo
        map_nome_atividade = row['nome_atividade']
        
        map_status_documento = row['status_documento']
        map_original_file_name = row['original_file_name']
        map_directory = row['directory']
        
        map_one_page = row['one_page']
        
        map_palavra_chave = row['palavra_chave']
        map_document_tag = row['document_tag']
        map_action_item = row['action_item']
        
        map_level = row['level']
        map_file_path = row['file_path']
        
        dados_iniciais['document_unique_id'] = map_document_unique_id
        
        file_path = map_file_path

        if (map_status_documento != 'NO_PROCESS'):
            #1. Buscar Prefeitura, de/para e modelo
            if map_status_documento == 'PREPROCESS_EXTRACT':
                
                dados_iniciais = processar_dados_iniciais(row, map_original_file_name, map_file_path)
                
                
                prefeitura_map = dados_iniciais['prefeitura']
                pdf_pesquisavel_map = dados_iniciais['pdf_pesquisavel']
                de_para_map = dados_iniciais['de_para_pm']
                model_map = dados_iniciais['model']
                #cnpj_map = dados_iniciais['cnpj']
            
                row_info['prefeitura'] = prefeitura_map
                row_info['pdf_pesquisavel'] = pdf_pesquisavel_map    
                row_info['de_para_pm'] = de_para_map
                row_info['model'] = model_map
                row_info['model'] = model_map
                #row_info['cnpj'] = cnpj_map
                data_cabecalho = {}
                data_cabecalho['secao'] = "1 - CABECALHO"
                
                if pdf_pesquisavel_map:
                   data_cabecalho = processar_cabecalho_pdf_pesquisavel(row, pdf_pesquisavel_map, model_map, map_original_file_name, file_path)
                   
                   nome_prefeitura_map = data_cabecalho['nome_prefeitura'] 
                   tipo_nota_fiscal_map = data_cabecalho['tipo_nota_fiscal'] 
                   nro_nota_map = data_cabecalho['numero_nota_fiscal']
                   competencia_map = data_cabecalho['competencia']
                   dt_hr_emissao_map = data_cabecalho['dt_hr_emissao']  
                   codigo_verificacao_map = data_cabecalho['codigo_verificacao']   
                   
                   
                else:
                    data_cabecalho = processar_cabecalho_raster_pdf(row, pdf_pesquisavel_map, model_map, map_original_file_name, file_path)
                    nome_prefeitura_map = data_cabecalho['nome_prefeitura']
                    tipo_nota_fiscal_map = data_cabecalho['tipo_nota_fiscal']
                    nro_nota_map = data_cabecalho['numero_nota_fiscal']
                    competencia_map = data_cabecalho['competencia']
                    dt_hr_emissao_map = data_cabecalho['dt_hr_emissao']  
                    codigo_ver_map = data_cabecalho['codigo_verificacao']
                    
                row_teste_info.append(data_cabecalho)        
                
                
                # print(f' 1 - seq: {map_seq:>5} Pesquisa? {pdf_pesquisavel_map} | {nro_nota_frame} | Pesq: {competencia_frame:>10} | dt_hr_emissao: {dt_hr_emissao_frame} | cod: {codigo_ver_frame} | file: {map_original_file_name} ')
                  

                lista_dicts.append(row_info)

        i += 1

    novo_df = pd.DataFrame(lista_dicts) 
                   
              
    return novo_df

In [60]:
# 3B. XXX Efetuo a extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'PREPROCESS' 
status = 'PREPROCESS_EXTRACT'
raw_document_list = []
dados_iniciais_qr = {}

#raw_texto = extracao_pipeline(df_analise_pipe, fase, atividade, status)subset_df
# df_ini, dados_iniciais_qr = extracao_pipeline(df_root_pipe, fase, atividade, status)
df = extracao_pipeline(df_root_pipe, fase, atividade, status)

In [61]:
df

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,...,action_item,level,parent_document_unique_id,file_hash,file_path,informations,prefeitura,pdf_pesquisavel,de_para_pm,model
0,7,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Nota Fiscal Eletrônica julho.pdf,1004268,False,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,0c9302206ff324aac606554ecfa215cb7c955929ca4b8b...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,True,PM_MAGE,MAGE
1,14,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,1.pdf,11790236,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,66a7db9ee1500d5f9fa5da26563cfd7b68f1f5ba3daba2...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
2,18,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,NF 55 JML.pdf,1004813,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,c6fc93ab85e40f4804b1129827e96d087e0efcf09b4b64...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
3,22,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Digitalização 26 de jul de 2023(1).pdf,11779446,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,6a96cd40e1a2f525a27e564b31099bda7d4ec035db62a6...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
4,23,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,JULH-2023-NotaFiscal 2.pdf,241538,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,32016344bc6b954bc69fe5b065a93cb6261be8bfbc06b7...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
5,24,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Nota Fiscal Nº27 VITALE ECO.pdf,11778530,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,00c372e293ffdfb8c04e984458940eaf265b8c5968e20f...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,True,PM_MAGE,MAGE
6,31,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,NFS.300 07.2023 FLOC TEXTIL.pdf,10597,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,b2096003fcbd7e9c75aa782256d366793e22bb4b4aebfd...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,True,PM_MAGE,MAGE


In [56]:
df.set_index('document_unique_id', inplace=True)

KeyError: "None of ['document_unique_id'] are in the columns"

In [50]:
df

,seq,date_time,batch,fase_processo,nome_atividade,status_documento,acao_executada,original_file_name,directory,one_page,...,action_item,level,parent_document_unique_id,file_hash,file_path,informations,prefeitura,pdf_pesquisavel,de_para_pm,model
0,7,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Nota Fiscal Eletrônica julho.pdf,1004268,False,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,0c9302206ff324aac606554ecfa215cb7c955929ca4b8b...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,True,PM_MAGE,MAGE
1,14,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,1.pdf,11790236,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,66a7db9ee1500d5f9fa5da26563cfd7b68f1f5ba3daba2...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
2,18,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,NF 55 JML.pdf,1004813,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,c6fc93ab85e40f4804b1129827e96d087e0efcf09b4b64...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
3,22,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Digitalização 26 de jul de 2023(1).pdf,11779446,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,6a96cd40e1a2f525a27e564b31099bda7d4ec035db62a6...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
4,23,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,JULH-2023-NotaFiscal 2.pdf,241538,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,32016344bc6b954bc69fe5b065a93cb6261be8bfbc06b7...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,False,PM_MAGE,MAGE
5,24,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,Nota Fiscal Nº27 VITALE ECO.pdf,11778530,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,00c372e293ffdfb8c04e984458940eaf265b8c5968e20f...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,True,PM_MAGE,MAGE
6,31,13/09/2023 08:32:22,Batch_20,analise,scan_analise,PREPROCESS_EXTRACT,Analise,NFS.300 07.2023 FLOC TEXTIL.pdf,10597,True,...,PREPROCESS_EXTRACT,3,67ea057e-09ab-4cc1-9ca5-6a80f15b260c,b2096003fcbd7e9c75aa782256d366793e22bb4b4aebfd...,pipeline_extracao_documentos/2_documentos_para...,NaN,PREFEITURA MUNICIPAL DE MAGE,True,PM_MAGE,MAGE


In [38]:
# 10. Consulta multiparametros
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id

In [ ]:
# Pesquisando os dados para o processo de extracao - Raster

#f_type = 'frame'
f_type = 'field_box'

filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['de_para_pm'] == de_para_pm) & (frames_nf_v4_df['model'] == model2search) & (frames_nf_v4_df['type'] == f_type)]

for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    seq = row_frame['seq']
    de_para = row_frame['de_para_pm']
    model = row_frame['model']  
    print(f'{seq:>4} {de_para:>8} | {model:>10} | {row_frame["type"]:>15} | {row_frame["label"]:>35} | {row_frame["section_json"]:>35} | {row_frame["x0"]:>7} | {row_frame["y0"]:>7} | {row_frame["x1"]:>7} | {row_frame["y1"]:>7}')


In [ ]:
# Pesquisando os dados para o processo de extracao - PDF Pesquisavel

f_type = 'frame'

filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['de_para_pm'] == de_para_pm) & (frames_nf_v4_df['model'] == model2search) & (frames_nf_v4_df['type'] == f_type)]


for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    seq = row_frame['seq']
    de_para = row_frame['de_para_pm']
    model = row_frame['model']  
    print(f'{seq:>4} {de_para:>8} | {model:>10} | {row_frame["type"]:>15} | {row_frame["label"]:>35} | {row_frame["section_json"]:>35} | {row_frame["x0_p"]:>7} | {row_frame["y0_p"]:>8} | {row_frame["x1_p"]:>8} | {row_frame["y1_p"]:>8}')

In [ ]:
filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model2search) & (frames_nf_v4_df['type'] == tipo)]

In [27]:
# 1. XXX Avaliar Funçao basica de modelo 
def pesquisa_model_frame_coordenadas(model_src, tipo, tipo_processo_pdf):

    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        frame_prefeitura = row_frame['prefeitura']
        frame_label = row_frame['label']
        frame_section_json = row_frame['section_json']
        frame_cnpj = row_frame['cnpj']
        
        if frame_model:
            modelo = True
        
        if tipo_processo_pdf == 'pdf_pesquisavel':
            try:
                # data for PDF Pesquisavel
                x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
                
                data_dados_model = {
                    "prefeitura": frame_prefeitura,
                    "model": frame_model,
                    "label": frame_label,
                    "section_json": frame_section_json,
                    "cnpj": frame_cnpj,
                    "x0": x0,
                    "y0": y0,
                    "x1": x1,
                    "y1": y1,
                }
                dados_modelo.append(data_dados_model)   
        
            except Exception as e:
                print(f"Erro ao carregar coordenadas do frame: {e}")
                x0, y0, x1, y1 = 0, 0, 0, 0 
                

        elif tipo_processo_pdf == 'raster_pdf':
            try:
                # data for Raster_PDF
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                
                data_dados_model = {
                    "prefeitura": frame_prefeitura,
                    "model": frame_model,
                    "label": frame_label,
                    "section_json": frame_section_json,
                    "cnpj": frame_cnpj,
                    "x0": x0,
                    "y0": y0,
                    "x1": x1,
                    "y1": y1,
                }
                dados_modelo.append(data_dados_model)
            except Exception as e:
                print(f"Erro ao carregar coordenadas do frame: {e}")
                x0, y0, x1, y1 = 0, 0, 0, 0 
                  
  
    return dados_modelo, modelo

In [28]:
# 1. funçao basica de modelo 
def pesquisa_model_frame(model_src, tipo):

    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        # frame_prefeitura = row_frame['prefeitura']
        # frame_label = row_frame['label']
        # frame_section_json = row_frame['section_json']
        # frame_cnpj = row_frame['cnpj']
        
        # if frame_model:
        #     modelo_frame = True
        # else:    
        #     modelo_frame = False

                  
  
    return frame_model

In [29]:
# 1. XXX funçao basica de modelo 
def pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label):

    tipo = tipo
    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['label'] == label)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        frame_label = row_frame['label']
 
        # data for PDF Pesquisavel
        x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
 
    return x0, y0, x1, y1

In [30]:
# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None

# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None
    
def extract_fields_cabecalho_raster_pdf(row, file_path, image_2work): 
  
    message_erro = []
  
  
    model_src = row['modelo']
    original_file_name = row['original_file_name'] 
    
    model = row['modelo']   
    secao = "1 - CABECALHO"
    section = "1 - CABECALHO"
    f_frame_name = "1_frame_dados_nf"
    f_tipo = 'frame'
        
    raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) # executa_model_frame
    
    
    try:
        text_splited = texto_extraido_cabecalho(raw_texto_pil)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
        
    except Exception as e:
        # erros_cabecalho = {}
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
        
        
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section, raw_texto_pil 

def extract_fields_cabecalho_pdf_pesquisavel(row, file_path):
    
    message_erro = [] 
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    
    
    model_src = row['modelo']
    original_file_name = row['original_file_name']
    tipo = "frame"
    section = '1 - CABECALHO'
    label = '1_frame_prefeitura_nf'

    x0, y0, x1, y1 = pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label)

    try:
        texto = page.get_text("text", clip=(x0, y0, x1, y1))
        
        text_splited = texto_extraido_nf(texto)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
            
    pdf_document.close()
    #print(row)
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section


# 5. XXX Ajusta texto
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

# 6. XXX Ajusta texto para PDF_Pesquisavel-  NO CABECALHO
def texto_extraido_nf(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


# 6. XXX Ajusta texto para PDF RASTER NO CABECALHO
def texto_extraido_cabecalho(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    text_splited = [s.replace(")", "").strip() for s in text_splited] #depende da situaçao
    return text_splited
    

In [ ]:
dict_raw = {}
i = 0
for  doc in raw_document_list:
    dict_raw = raw_document_list[i]
    print(i)
    print(dict_raw['pdf_pesquisavel'])
    print(dict_raw['original_file_name'])
    print(dict_raw['texto_extraido'])
    #print(f'\nraw_document_list[i]: {raw_document_list[i]}\n\n')
    i += 1

In [31]:
def extract_fields_cnpj(texto_extraido, original_file_name): 
  
    message_erro = []
    
    data_nf_cnpj_prestador = {}
      
    print(f'2 extract_fields_cnpj:  {texto_extraido}')
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in texto_extraido:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', texto_extraido)
        if cpf_cnpj_formatado_match:
            data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
            data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
    else:
        data_nf_cnpj_prestador['cpf_cnpj_com_mascara'] = None
        data_nf_cnpj_prestador['cpf_cnpj_sem_mascara'] = None            
            
            
    telefone_str = None

    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', texto_extraido)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        data_nf_cnpj_prestador['telefone'] = telefone_str
    else:
        data_nf_cnpj_prestador['telefone'] = None 
    
    #print(data_nf_cnpj_prestador)    

             
    return data_nf_cnpj_prestador

In [ ]:
import pandas as pd

# Criando um DataFrame de exemplo
data = {'A': [1, 2], 'B': [3, 4], 'C': [5, 6]}
df = pd.DataFrame(data)

# Criando uma lista para armazenar os dicionários que representam cada linha
lista_dicts = []

# Iterando sobre as linhas do DataFrame e adicionando cada linha à lista
for index, row in df.iterrows():
    # Convertendo a linha a um dicionário
    row_dict = row.to_dict()
    
    # Adicionando novas colunas ao dicionário
    row_dict['D'] = row['A'] * row['B']
    row_dict['E'] = row['B'] + row['C']
    
    # Adicionando o dicionário à lista
    lista_dicts.append(row_dict)

# Criando um novo DataFrame a partir da lista de dicionários
novo_df = pd.DataFrame(lista_dicts)

# Exibindo o novo DataFrame
print(novo_df)